### 1. 简介

- 2DPCA基于二维图像矩阵而不是一维向量，因此在特征提取前不需要将图像矩阵转换为向量。
- 2DPCA直接使用原始图像矩阵构建图像协方差矩阵，并通过其特征向量进行图像特征提取。
- 实验表明识别率和提取效率都比PCA更高

### 2. 算法流程

#### 2.1 思想与算法
设 $X$ 表示一个 $n$ 维单位列向量。我们的思想是通过以下线性变换将图像 $A$（一个 $m \times n$ 的随机矩阵）投影到 $X$ 上：

$$\mathbf{Y}=\mathbf{AX}$$
(1)
因此，我们得到一个 $m$ 维的投影向量 $Y$，称为图像 $A$ 的投影特征向量。如何确定一个好的投影向量 $X$ 呢？事实上，投影样本的总散布可以用来衡量投影向量 $X$ 的判别能力。投影样本的总散布可以用投影特征向量的协方差矩阵的迹来表示。从这个角度，我们采用以下准则：
$$J(\mathbf{X})=\mathrm{tr}(\mathbf{S}_x),$$ 
(2)
其中 $\mathbf{S}_x$ 表示训练样本的投影特征向量的协方差矩阵，$\mathrm{tr}(\mathbf{S}_x)$ 表示 $\mathbf{S}_x$ 的迹。最大化准则 (2) 的物理意义是找到一个投影方向 $X$，使得所有样本投影后的总散布最大。协方差矩阵 $\mathbf{S}_x$ 可以表示为：
$$\begin{aligned}\mathbf{S}_{x}&=E(\mathbf{Y}-E\mathbf{Y})(\mathbf{Y}-E\mathbf{Y})^T=E[\mathbf{A}\mathbf{X}-E(\mathbf{A}\mathbf{X})][\mathbf{A}\mathbf{X}-E(\mathbf{A}\mathbf{X})]^T\\&=E[(\mathbf{A}-E\mathbf{A})\mathbf{X}][(\mathbf{A}-E\mathbf{A})\mathbf{X}]^T.\end{aligned}$$
因此，
$$\mathrm{tr}(\mathbf{S}_x)=\mathbf{X}^T[E(\mathbf{A}-E\mathbf{A})^T(\mathbf{A}-E\mathbf{A})]\mathbf{X}.$$ 
(3)
定义如下矩阵：
$$\mathbf{G}_t=E[(\mathbf{A}-E\mathbf{A})^T(\mathbf{A}-E\mathbf{A})],$$ 
(4)
矩阵 $\mathbf{G}_t$ 称为图像协方差（散布）矩阵。根据定义可以很容易验证 $\mathbf{G}_t$ 是一个 $n \times n$ 的非负定矩阵。我们可以直接使用训练图像样本来计算 $\mathbf{G}_t$。假设共有 $M$ 个训练图像样本，第 $j$ 个训练图像表示为一个 $m \times n$ 的矩阵 $\mathbf{A}_{j}(j=1,2,\cdots,M)$，所有训练样本的平均图像表示为 $\bar{\mathbf{A}}$。然后 $\mathbf{G}_t$ 可以表示为：
$$\mathbf{G}_t=\dfrac{1}{M}\sum_{j=1}^M\left(\mathbf{A}_j-\bar{\mathbf{A}}\right)^T(\mathbf{A}_j-\bar{\mathbf{A}}).$$ 
(5)
因此，准则 (2) 可以表示为：
$$J(\mathbf{X})=\mathbf{X}^T\mathbf{G}_t\mathbf{X},$$ 
(6)
其中 $X$ 是一个单位列向量。这个准则称为广义总散布准则。最大化该准则的单位向量 $X$ 称为最优投影轴。直观上，这意味着在将图像矩阵投影到 $X$ 上后，投影样本的总散布最大。最优投影轴 $X_{\text{opt}}$ 是最大化 $J(\mathbf{X})$ 的单位向量，即 $\mathbf{G}_t$ 的最大特征值对应的特征向量[19]。一般来说，仅有一个最优投影轴是不够的。我们通常需要选择一组投影轴 $\mathbf{X}_1, \cdots, \mathbf{X}_d$，使其满足正交归一约束并最大化准则 $J(\mathbf{X})$，即：
$$\left\{\begin{array}{l}\{\mathbf{X}_1,\cdots,\mathbf{X}_d\}=\arg\max J(\mathbf{X})\\\mathbf{X}_i^T\mathbf{X}_j=0,i\neq j,i,j=1,\cdots,d.\end{array}\right.$$
(7)
事实上，最优投影轴 $\mathbf{X}_1, \cdots, \mathbf{X}_d$ 是对应于 $\mathbf{G}_t$ 的前 $d$ 个最大特征值的正交归一特征向量。
#### 2.2 特征提取
2DPCA 的最优投影向量 $\mathbf{X}_1, \cdots, \mathbf{X}_d$ 用于特征提取。对于给定的图像样本 $A$，设
$$\mathbf{Y}_k=\mathbf{A}\mathbf{X}_k,k=1,2,\cdots,d.$$ 
(8)
然后，我们得到一组投影特征向量 $\mathbf{Y}_1, \cdots, \mathbf{Y}_d$，称为样本图像 $A$ 的主成分（向量）。需要注意的是，2DPCA 的每个主成分是一个向量，而 PCA 的主成分是一个标量。获得的主成分向量用于形成一个 $m \times d$ 的矩阵 $\mathbf{B}=[\mathbf{Y}_1, \cdots, \mathbf{Y}_d]$，称为图像样本 $A$ 的特征矩阵或特征图像。
#### 2.3 分类方法
通过2DPCA变换后，每个图像获得一个特征矩阵。然后使用最近邻分类器进行分类。这里，任意两个特征矩阵 $\mathbf{B}_i=[\mathbf{Y}_1^{(i)}, \mathbf{Y}_2^{(i)}, \cdots, \mathbf{Y}_d^{(i)}]$ 和 $\mathbf{B}_j=[\mathbf{Y}_1^{(j)}, \mathbf{Y}_2^{(j)}, \cdots, \mathbf{Y}_d^{(j)}]$ 之间的距离定义为：
$$d(\mathbf{B}_i, \mathbf{B}_j)=\sum_{k=1}^d \left\|\mathbf{Y}_k^{(i)} - \mathbf{Y}_k^{(j)}\right\|_2,$$ 
(9)
其中 $||\mathbf{Y}_k^{(i)} - \mathbf{Y}_k^{(j)}||_2$ 表示两个主成分向量 $\mathbf{Y}_k^{(i)}$ 和 $\mathbf{Y}_k^{(j)}$ 之间的欧几里得距离。假设训练样本为 $\mathbf{B}_1, \mathbf{B}_2, \cdots, \mathbf{B}_M$（其中 $M$ 是训练样本总数），每个样本被分配一个给定的身份（类）$\omega_k$。给定测试样本 $\mathbf{B}$，如果 $d(\mathbf{B}, \mathbf{B}_l) = \min_j d(\mathbf{B}, \mathbf{B}_j)$ 且 $\mathbf{B}_l \in \omega_k$，则决策结果为 $\mathbf{B} \in \omega_k$。
#### 3 2DPCA基于图像重构
在 Eigenfaces 方法中，主成分和特征向量（特征脸）可以组合起来重构人脸图像。类似地，2DPCA 可以通过以下方式重构人脸图像。假设图像协方差矩阵 $\mathbf{G}_t$ 的前 $d$ 个最大特征值对应的正交特征向量为 $\mathbf{X}_1, \cdots, \mathbf{X}_d$。图像样本投影到这些轴上，得到的主成分向量为 $\mathbf{Y}_k=\mathbf{AX}_k$ $(k=1,2,\cdots,d)$。设 $\mathbf{V}=[\mathbf{Y}_1, \cdots, \mathbf{Y}_d]$ 和 $\mathbf{U}=[\mathbf{X}_1, \cdots, \mathbf{X}_d]$，则
$$\mathbf{V}=\mathbf{AU}.$$ 
(10)
由于 $\mathbf{X}_1, \cdots, \mathbf{X}_d$ 是正交归一的，从 (10) 可以很容易得到样本 $A$ 的重构图像：
$$\tilde{\mathbf{A}}=\mathbf{VU}^T=\sum_{k=1}^d \mathbf{Y}_k \mathbf{X}_k^T.$$ 
(11)
设 $\tilde{\mathbf{A}}_k=\mathbf{Y}_k \mathbf{X}_k^T (k=1,2,\cdots,d)$，它的大小与图像 $A$ 相同，表示 $A$ 的重构子图像。也就是说，图像 $A$ 可以通过叠加前 $d$ 个子图像进行近似重构。特别是，当选择的主成分向量数量 $d=n$（$n$ 是 $\mathbf{G}_t$ 的特征向量总数）时，我们有 $\tilde{\mathbf{A}}=\mathbf{A}$，即图像通过其主成分向量完全重构而没有任何信息损失。否则，如果 $d<n$，重构图像 $\tilde{\mathbf{A}}$ 是 $A$ 的近似值。

### 3. 摘译

#### 3.1. 摘要

本文开发了一种新的技术，称为二维主成分分析（2DPCA），用于图像表示。与PCA不同，2DPCA基于二维图像矩阵而不是一维向量，因此在特征提取之前不需要将图像矩阵转换为向量。相反，2DPCA直接使用原始图像矩阵构建图像协方差矩阵，并通过其特征向量进行图像特征提取。为了测试2DPCA并评估其性能，我们在三个面部图像数据库（ORL、AR和Yale面部数据库）上进行了一系列实验。所有试验中的识别率均表明使用2DPCA比PCA更高。实验结果还表明，2DPCA在图像特征提取方面比PCA具有更高的计算效率。


